In [22]:
import numpy as np

import mne

from mne.preprocessing import ICA
import asrpy
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, LeaveOneOut, LeavePOut

import matplotlib.pyplot 

# Chemin vers les fichiers EEG et les positions des électrodes
raw_fnames = 'C:/recordings/Game_recordings_test/TOUT_4_second_apple/mergeddata.set'
montage_fname = 'path/to/montage_file.elc'  # Remplacez par le chemin vers votre fichier de montage

# Charger les données EEG
raw = mne.io.read_raw_eeglab(raw_fnames, preload=True)

# Charger et définir le montage
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)




Reading C:\recordings\Game_recordings_test\TOUT_4_second_apple\mergeddata.fdt
Reading 0 ... 809559  =      0.000 ...  3238.236 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


C:\Users\robinaki\AppData\Local\Temp\ipykernel_19140\1064415564.py:22: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(raw_fnames, preload=True)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [23]:
new_srate = 25
tmin = -0.5  # début de chaque époque (1,0 secondes avant le déclencheur)
tmax = 2  # fin de chaque époque (au déclencheur)
baseline = (None, 0)  # baseline du début à t = 0
reject = dict(eeg=150e-3)  # seuil de rejet pour les électrodes EEG
L_H_freq = [(8,13),(13,30)]
Notchs_freq = (50,60)
f_low = 1
raw.filter( f_low, None)
raw.notch_filter(Notchs_freq)
raw.filter(8, 30)
# Appliquer ASR pour nettoyer les artefacts
asr = asrpy.ASR(sfreq=raw.info["sfreq"], cutoff=20)
asr.fit(raw)
raw = asr.transform(raw)
# ica = ICA()
# ica.fit(raw)
# raw= ica.apply(raw)
# raw.resample(new_srate)

events , event_id= mne.events_from_annotations(raw)
event_id_RL= {'right' : event_id['right'] , 'left' : event_id['left']}
epochs = mne.Epochs(raw, events, event_id_RL, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
info = epochs.info

# Calculer la matrice de covariance de bruit
noise_cov = mne.make_ad_hoc_cov(info)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 s)



Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descri

In [24]:
# Utiliser un modèle de tête standard fourni par MNE
subjects_dir = 'C:/Users/robinaki/mne_data/MNE-sample-data/subjects'


# noise_cov = mne.make_ad_hoc_cov(info)
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)
bem = mne.read_bem_solution(
'C:/Users/robinaki/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif'
    )
src = mne.setup_source_space("fsaverage", spacing="oct6", add_dist="patch", subjects_dir=subjects_dir)
fwd = mne.make_forward_solution(
        info,
        trans="fsaverage",
        src=src,
        bem=bem,
        meg=False,
        eeg=True,
        mindist=5.0,
        n_jobs=None,
        verbose=False,
    )





0 files missing from root.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects
0 files missing from bem.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage
Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Setting up the source space with the following parameters:

SUBJECTS_DIR = C:\Users\robinaki\mne_data\MNE-sample-data\subjects
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.sphere...
Setting up the triangulation for th

In [25]:

# Calculer la solution inverse

inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8)
# stc = mne.minimum_norm.apply_inverse_raw(raw, inverse_operator, lambda2=1.0 / 9.0, method='dSPM')

# Solution of the inverse solution for evoked data (averaged data) by marker (event ID left and right)

event_id_R =  {'right' : event_id['right']}
event_id_L= {'left' : event_id['left']}
epochsR =  mne.Epochs(raw, events, event_id_R, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
epochsL =  mne.Epochs(raw, events, event_id_L, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)

evokedL = epochsL.average()
evokedR = epochsR.average()


stcR, residualR = mne.minimum_norm.apply_inverse(
    evokedR,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='dSPM',
    pick_ori=None,
    return_residual=True,
    verbose=True,
)
stcL, residualR = mne.minimum_norm.apply_inverse(
    evokedL,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='dSPM',
    pick_ori=None,
    return_residual=True,
    verbose=True,
)


Converting forward solution to surface orientation
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 8 channels.
    8 out of 8 channels remain after picking
Selected 8 channels
Creating the depth weighting matrix...
    8 EEG channels
    limit = 8197/8196 = 5.326282
    scale = 7695.39 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 7.1e-19 (2.2e-16 eps * 8 dim * 0.0004  max singular value)
    Estimated rank (eeg): 7
    EEG: rank 7 computed from 8 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
   

In [29]:
vertno_max, time_max = stcR.get_peak(hemi="rh")

surfer_kwargs = dict(
    hemi="lh",
    subjects_dir=subjects_dir,
    clim=dict(kind="value", lims=[8, 12, 15]),
    views="lateral",
    initial_time=time_max,
    time_unit="s",
    size=(800, 800),
    smoothing_steps=10,
)
brain = stcR.plot(**surfer_kwargs)
brain.add_foci(
    vertno_max,
    coords_as_verts=True,
    hemi="rh",
    color="blue",
    scale_factor=0.6,
    alpha=0.5,
)
brain.add_text(
    0.1, 0.9, "dSPM (plus location of maximal activation)", "title", font_size=14
)

# The documentation website's movie is generated with:
# brain.save_movie(..., tmin=0.05, tmax=0.15, interpolation='linear',
#                  time_dilation=20, framerate=10, time_viewer=True)

KeyError: 'rh'

In [27]:
raw.plot()

Channels marked as bad:
none
